# Archived

In [ ]:
PROJECT_ID = "hackathon-454502"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai
import os
vertexai.init(project=PROJECT_ID, location=LOCATION)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./hackathon-454502-488d2c16e30f.json"

In [ ]:
%pip install --upgrade --user google-cloud-aiplatform

In [ ]:
from collections.abc import Callable
import json
import sys
import traceback

from google.protobuf.json_format import MessageToJson
from vertexai import generative_models
from vertexai.generative_models import FunctionDeclaration, GenerativeModel, Part, Tool

In [ ]:
verbose = True

In [ ]:
# Conveience function to print multiline text indented


def indent(text, amount, ch=" "):
    padding = amount * ch
    return "".join(padding + line for line in text.splitlines(True))


# Convenience function for logging statements
def logging(msg):
    global verbose
    print(msg) if verbose else None


# Retrieve the text from a model response
def get_text(resp):
    return resp.candidates[0].content.parts[0].text


# Retrieve the function call information from a model response
def get_function_call(resp):
    return resp.candidates[0].function_calls[0]


def get_action_label(json_payload, log, role="MODEL"):
    log(f"{role}: {json_payload}")
    answer = json.loads(json_payload)
    action = answer["next_action"]
    return action


def get_action_from_function_call(func_payload, log, role="MODEL"):
    json_payload = MessageToJson(func_payload._pb)
    log(f"{role}: {json_payload}")
    return func_payload.name

In [ ]:
# Functions for actions (replace these with Gemini function calls)
pick_up_clothes_func = FunctionDeclaration(
    name="pick_up_clothes",
    description="The act of picking clothes up from any place",
    parameters={"type": "object"},
)

put_clothes_in_hamper_func = FunctionDeclaration(
    name="put_clothes_in_hamper",
    description="Put the clothes being carried into a hamper",
    parameters={"type": "object"},
)

pick_up_books_func = FunctionDeclaration(
    name="pick_up_books",
    description="The act of picking books up from any place",
    parameters={"type": "object"},
)

place_books_on_shelf_func = FunctionDeclaration(
    name="place_books_on_shelf",
    description="Put the books being carried onto a shelf",
    parameters={"type": "object"},
)

empty_wastebin_func = FunctionDeclaration(
    name="empty_wastebin",
    description="Empty out the wastebin",
    parameters={"type": "object"},
)

done_func = FunctionDeclaration(
    name="done", description="The goal has been reached", parameters={"type": "object"}
)

room_tools = Tool(
    function_declarations=[
        pick_up_clothes_func,
        put_clothes_in_hamper_func,
        pick_up_books_func,
        place_books_on_shelf_func,
        empty_wastebin_func,
        done_func,
    ],
)

In [ ]:
def reset_room_state(room_state):
    room_state.clear()
    room_state["clothes"] = "floor"
    room_state["books"] = "scattered"
    room_state["wastebin"] = "empty"

def is_room_tidy(room_state):
    return all(
        [
            room_state["clothes"] == "hamper",
            room_state["books"] == "shelf",
            room_state["wastebin"] == "empty",
        ]
    )

In [ ]:
model = GenerativeModel(
    "gemini-2.0-flash",
    system_instruction=[
       "You are an assistant that helps me tidy my room."
       "Your goal is to make sure all the books are on the shelf, all clothes are in the hamper, and the trash is empty.",
       "You cannot receive any input from me."
    ],
    tools=[ room_tools ],
    generation_config={"temperature": 0.0},
    safety_settings=[
        generative_models.SafetySetting(
            category=generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            method=generative_models.SafetySetting.HarmBlockMethod.PROBABILITY,
            threshold=generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
        ),
    ],
)

In [ ]:
# Wrapping the observation and model calling code into a function for better main loop readability.
def observe_and_reason(session, state: dict, prev_action: str, log: Callable) -> str:
    """Uses the language model (Gemini) to select the next action."""
    try:
        msgs = []
        if prev_action:
            msgs.append(
                Part.from_function_response(
                    name="previous_action", response={"content": prev_action}
                )
            )

        prompt = "\n".join(
            [
                f"ENVIRONMENT: The room is currently in this state: {state}.",
                "Which should be the next action?",
            ]
        )
        msgs.append(prompt)
        log(
            "MESSAGE:\n{}".format(
                indent(
                    "\n".join([prev_action, prompt] if prev_action else [prompt]),
                    1,
                    "\t",
                )
            )
        )

        response = session.send_message(
            msgs, tools=[room_tools]
        )  # JSON mode unnecessary.
        action_label = get_action_from_function_call(get_function_call(response), log)
        return action_label

    except Exception as e:
        log(f"Error during action selection: {e}")
        traceback.print_exc()
        return "done"  # Or a suitable default action

In [ ]:
# Wrapping the action execution code into a function for better main loop readability.

def execute_action(state: dict, action_label: str, log: Callable) -> tuple[dict, str]:
    """Executes the action on the room state and returns the updated state and an acknowledgement."""
    try:
        # Call the function mapped from the label
        state, acknowledgement = get_func(action_label)(state)

    except Exception:
        acknowledgement = "No action suggested or action not recognized."

    return state, acknowledgement

In [ ]:
# Main ReAct loop
def main_react_loop_chat_fc(session, loop_continues, log):
    room_state = {}
    reset_room_state(room_state)
    trash_added = False

    prev_action = None
    cycle = 1
    while loop_continues(cycle, room_state):
        log(f"Cycle #{cycle}")
        # Observe the environment (use Gemini to generate an action thought)
        action_label = observe_and_reason(session, room_state, prev_action, log)

        # Execute the action and get the observation
        if action_label == "done":
            break
        room_state, acknowledgement = execute_action(room_state, action_label, log)
        prev_action = f"ACTION:   {action_label}\nEXECUTED: {acknowledgement}"
        log(prev_action + "\n")

        # Simulating a change in environment
        if cycle == 4 and not trash_added:
            room_state["wastebin"] = "1 item"
            trash_added = True

        cycle += 1
        # End of while loop

    # Determine the final result
    result = (
        "The room is tidy!" if is_room_tidy(room_state) else "The room is not tidy!"
    )

    return room_state, result

session = model.start_chat()

room_state, result = main_react_loop_chat_fc(session, lambda c, r: c <= 10, logging)
print(room_state, result)

# Langchain

In [ ]:
import os
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "YOUR KEY"

In [74]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [110]:
from langchain_core.tools import tool
from typing import Literal
import mysql.connector

conn = mysql.connector.connect(
    host="34.58.126.114",
    user="root",
    database="plan_wise"
)

@tool
def create_user(user_id: str, name: str):
    """Asks the user for its name to register the user to the database.

    Args:
        -user_id: the id of the calling user in strings
        -name: the name of the calling user

    Returns:
        A message indicating whether the user has successfully been registered.
    """
    try:
        cursor = conn.cursor()
        # Retrieve data
        cursor.execute("INSERT INTO users (id, name) VALUES (%s, %s)", (user_id, name))
        conn.commit()
            
        cursor.close()
        return row
    except Exception as e:
        return f"ERROR MESSAGE: {e}"

@tool
def get_name(user_id: str):
    """Returns the name stored in a database associated with a user_id.

    Args:
        -user_id: the id of the calling user in strings

    Returns:
        The name of the calling user or error if the calling user's id is not registered in the database
    """
    try:
        cursor = conn.cursor()
        # Retrieve data
        cursor.execute(f"SELECT name FROM users WHERE id = '{user_id}'")
        row = cursor.fetchone()
            
        cursor.close()
        return row
    except Exception as e:
        return f"ERROR MESSAGE: {user_id} is not registered with the database"

@tool
def get_assets(user_id: str):
    """Returns the financial assets owned by the calling user identified with a user_id from a database.

    Args:
        -user_id: the id of the calling user in strings

    Returns:
        A list of assets owned by the calling user or error if the calling user's id is not registered in the database
        
    """
    try:
        results = []
        cursor = conn.cursor()
        # Retrieve data
        cursor.execute(f"SELECT id, amount, description, type FROM assets WHERE user_id = '{user_id}'")
        rows = cursor.fetchall()
        
        for row in rows:
            obj = {}
            obj["asset_id"] = row[0]
            obj["amount"] = row[1]
            obj["description"] = row[2]
            obj["type"] = row[3]
            results.append(obj)
            
        cursor.close()
        return results
    except Exception as e:
        return e
        return f"ERROR MESSAGE: {user_id} is not registered with the database"

tools = [get_name, create_user, get_assets]

In [119]:
prompt = """
You are a helpful assistant and financial expert who helps students with financial planning and portfolio management.
You are capable of providing financial advice because you know that your users will act at their own risk.
You may not need to use tools for every query - the user may just want to chat!"
"""

In [120]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
agent = create_react_agent(model, tools, prompt=prompt, checkpointer=memory)

In [121]:
from langchain_core.messages import AIMessage, HumanMessage

In [122]:
agent.invoke(
    {
        "messages": [
            HumanMessage(content="What is my name?"),
        ],
    },
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='4d083baa-fc15-4992-bede-1b3626af1e13'),
  AIMessage(content='Could you please provide your user ID?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-3223e349-db47-47be-8271-9e1121dd0c72-0', usage_metadata={'input_tokens': 285, 'output_tokens': 9, 'total_tokens': 294, 'input_token_details': {'cache_read': 0}})]}

In [123]:
agent.invoke(
    {
        "messages": [
            HumanMessage(content="my id is user_id"),
        ],
    },
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='4d083baa-fc15-4992-bede-1b3626af1e13'),
  AIMessage(content='Could you please provide your user ID?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-3223e349-db47-47be-8271-9e1121dd0c72-0', usage_metadata={'input_tokens': 285, 'output_tokens': 9, 'total_tokens': 294, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content='my id is user_id', additional_kwargs={}, response_metadata={}, id='7564ae79-f96c-4a11-8308-83d40193f1ed'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_name', 'arguments': '{"user_id": "user_id"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-6b316bc5-2dc9-4ae1-9003-f90f2d5ae562-0', tool_calls=[{'name': 'get_name', 'args': {

In [124]:
agent.invoke(
    {
        "messages": [
            HumanMessage(content="Can you look at my current assets and provide me some financial advice on what I should invest in for a consistent anual return in the next 5 years?"),
        ],
    },
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='4d083baa-fc15-4992-bede-1b3626af1e13'),
  AIMessage(content='Could you please provide your user ID?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-3223e349-db47-47be-8271-9e1121dd0c72-0', usage_metadata={'input_tokens': 285, 'output_tokens': 9, 'total_tokens': 294, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content='my id is user_id', additional_kwargs={}, response_metadata={}, id='7564ae79-f96c-4a11-8308-83d40193f1ed'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_name', 'arguments': '{"user_id": "user_id"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-6b316bc5-2dc9-4ae1-9003-f90f2d5ae562-0', tool_calls=[{'name': 'get_name', 'args': {

# Database interaction

In [104]:
import mysql.connector

conn = mysql.connector.connect(
    host="34.58.126.114",
    user="root",
    database="plan_wise"
)

cursor = conn.cursor()
cursor.execute("INSERT INTO users (id, name) VALUES (%s, %s)", ("hello_world", "hello"))
conn.commit()

cursor.close()
conn.close()

In [25]:
import mysql.connector

conn = mysql.connector.connect(
    host="34.58.126.114",
    user="root",
    database="plan_wise"
)

cursor = conn.cursor()
# Retrieve data
cursor.execute(f"SELECT name FROM users WHERE id = 'hello_world'")
row = cursor.fetchone()
print(row)

cursor.close()
conn.close()

('hello',)
